In [2]:
from transformers import (
    BitsAndBytesConfig,
    LlavaNextProcessor,
    LlavaNextForConditionalGeneration
)
from PIL import Image
import torch
import pandas as pd
import re
import gc
from tqdm import tqdm
import os
import random

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True
)

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf", dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto", quantization_config=bnb_config
).to(device)

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

processor_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def clean_instruction_tokens(text):
    cleaned_text = re.sub(r'\[INST\]\s*\n?.*?\[/INST\]\s*', '', text, flags=re.DOTALL)
    return cleaned_text.strip()


def mllm_testing(df, processor, model, most="True"):
    with torch.inference_mode():
        torch.cuda.empty_cache()
        gc.collect()
        generated_texts = []
        for idx, row in df.iterrows():
            instruction_tokens = "[INST] <image>\n"
            end_tokens = "[/INST]"

            image_type = row['image_type']
            object_name = row['object']
            #question = f"What color is {'a' if most == 'True' else 'this'} {object_name}?"
            if most == "True":
                object_name_plural = object_name if object_name.endswith("s") else object_name + "s"
                question = f"What color are most {object_name_plural}?"

            else:
                question = f"What color is this {object_name}?"

            prompt = f"{instruction_tokens} Answer with one word. {question} {end_tokens}"

            try:
                image = Image.open(row['image_path']).convert("RGB")
                image = image.resize((256, 256), Image.LANCZOS)
            except FileNotFoundError:
                print(f"Warning: Image not found for {row['object']}")
                generated_texts.append(None)
                continue  # Skip to the next row in the DataFrame

            inputs = processor(images=image, text=prompt, return_tensors='pt')
            inputs = {k: v.to(device) for k, v in inputs.items()}
            # Perform a forward pass with the model
            outputs = model.generate(**inputs, max_new_tokens=10, num_beams=1, do_sample=False, temperature=1.0, pad_token_id=processor.tokenizer.eos_token_id)  # Adjust max_new_tokens as needed
            predicted_answer = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
            predicted_answer = clean_instruction_tokens(predicted_answer)

            generated_texts.append(predicted_answer)
            #print(torch.cuda.memory_summary())

            to_delete = ['inputs', 'outputs', 'image_inputs', 'video_inputs', 'generated_ids', 'prepare_inputs', 'image', 'pil_images', 'inputs_embeds']
            for var_name in to_delete:
                if var_name in locals():
                    var = locals()[var_name]
                    if isinstance(var, dict):
                        for v in var.values():
                            if torch.is_tensor(v) and v.is_cuda:
                                del v
                    elif torch.is_tensor(var) and var.is_cuda:
                        del var
                    del locals()[var_name]
            if hasattr(model, 'clear_kv_cache'):
                model.clear_kv_cache()
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            gc.collect()

            #print(torch.cuda.memory_summary())


        df['predicted_color'] = generated_texts

        if 'inputs' in locals(): del inputs
        if 'image' in locals(): del image
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        gc.collect()
    return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/Masterarbeit/")

In [ ]:
df = pd.read_parquet("data/fruit/fruit_images.parquet")

batch_size = 1
results = []

for i in tqdm(range(0, len(df), batch_size), desc="Running predictions"):
    batch_df = df.iloc[i:i + batch_size].copy()
    with torch.inference_mode():

        df_most = mllm_testing(batch_df, processor, model, most="True")
        df_most = df_most.rename(columns={"predicted_color": "pred_color_most"})

        df_this = mllm_testing(batch_df, processor, model, most="False")
        df_this = df_this.rename(columns={"predicted_color": "pred_color_this"})

        result_df = pd.merge(df_most, df_this, on=["image_path", "object", "image_type"])

    results.append(result_df)
    del result_df
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

df = pd.concat(results, ignore_index=True)
display(df)

df.to_csv(
    f'outline_results_new_blueberry.csv',
    index=False
)

Running predictions: 100%|██████████| 30/30 [10:19<00:00, 20.65s/it]


,object,image_type,image_path,pred_color_most,pred_color_this
0,banana,grayscale,data/fruit/banana1.jpeg,Yellow,Yellow
1,banana,background,data/fruit/banana1_background.jpeg,Yellow,White
2,banana,color,data/fruit/banana1_color.jpeg,Yellow,Yellow
3,banana,grayscale,data/fruit/banana2.jpeg,Yellow,Black
4,banana,background,data/fruit/banana2_background.jpeg,Yellow,White
5,banana,color,data/fruit/banana2_color.jpeg,Yellow,Yellow
6,blueberry,grayscale,data/fruit/blueberry.jpeg,White,White
7,blueberry,background,data/fruit/blueberry_background.jpeg,White,White
8,blueberry,color,data/fruit/blueberry_color.jpeg,Blue,Blue
9,cucumber,grayscale,data/fruit/cucumber.jpeg,Green,White
